### Import

In [1]:
import pubmed

### Download Neurosurgery articles

In [2]:
db = 'pubmed'
query = 'text classification neural network'
query0 = 'neurosurgery'
query1 = '(neurosurg*) AND (complicat*[Text Word])'
query2 = '("Glioma/surgery"[Mesh] OR "Glioma/therapy"[Mesh])'
query3 = '("Neurosurgery"[Mesh] OR "Neurosurgical Procedures"[Mesh]) AND "Postoperative Complications"[Mesh]'
query4 = '("Neurosurgery"[Mesh] OR "Neurosurgical Procedures"[Mesh]) AND complicat*[Text Word]'

In [3]:
article_ids = pubmed.download_all_query_responses(query=query, db=db, refresh=False)

237 articles found in pubmed with query specified.
0 articles are already stored in the database.
237 articles will be downloaded.



Total 237 articles stored in the database.


### Handle query responses

In [4]:
items = pubmed.handle_query_responses(db, article_ids)
items.head()

,pmid,pmc,pii,mid,doi,elocation_id,language,title,authors,article_type,...,publisher_nlm_id,publisher_issn_linking,abstract_len,abstract,copyright,mesh_quals_major,mesh_quals_minor,mesh_descriptors,keywords,file_size
0,32765261,PMC7378532,NaN,NaN,10.3389/fphar.2020.01028,10.3389/fphar.2020.01028,eng,Artificial Intelligence in Pharmacoepidemiolog...,Sessa Maurizio; Khan Abdul Rauf; Liang David; ...,pubmed_article,...,101548923,1663-9812,1496,Aim: To perform a systematic review on the app...,"Copyright © 2020 Sessa, Khan, Liang, Andersen ...",NaN,NaN,NaN,artificial intelligence; deep learning; machin...,33874
1,32764354,NaN,s20164369,NaN,10.3390/s20164369,E4369,eng,Earthquake Probability Assessment for the Indi...,Jena Ratiranjan; Pradhan Biswajeet; Al-Amri Ab...,pubmed_article,...,101204366,1424-8220,2212,Earthquake prediction is a popular topic among...,NaN,NaN,NaN,NaN,GIS; Indian subcontinent; deep learning; earth...,10267
2,32761550,NaN,10.1007/s11845-020-02332-1,NaN,10.1007/s11845-020-02332-1,10.1007/s11845-020-02332-1,eng,Machine learning for predicting long-term kidn...,Sekercioglu Nigar; Fu Rui; Kim S Joseph; Mitsa...,pubmed_article,...,7806864,0021-1265,1784,Supervised machine learning (ML) is a class of...,NaN,NaN,NaN,NaN,Chronic kidney allograft dysfunction; Risk pre...,7838
3,32729840,NaN,v8i7e17784,NaN,10.2196/17784,10.2196/17784,eng,Identifying and Predicting Intentional Self-Ha...,Obeid Jihad S; Dahne Jennifer; Christensen Sea...,pubmed_article,...,101645109,NaN,2309,BACKGROUND: Suicide is an important public hea...,"©Jihad S Obeid, Jennifer Dahne, Sean Christens...",NaN,NaN,NaN,deep learning; electronic health records; mach...,11313
4,32673791,NaN,S1532-0464(20)30139-8,NaN,10.1016/j.jbi.2020.103511,S1532-0464(20)30139-8,eng,An attention-based multi-task model for named ...,Wu Chaochen; Luo Guan; Guo Chao; Ren Yin; Zhen...,pubmed_article,...,100970413,1532-0464,794,"In this paper, we propose an attention-based m...",Copyright © 2020 Elsevier Inc. All rights rese...,NaN,NaN,NaN,Deep learning; Intent analysis; Interpretabili...,8424


In [33]:
items.to_csv('database/{0}_queryX.csv'.format(db), sep='|')
items.to_excel('database/{0}_queryX.xlsx'.format(db))